<a href="https://colab.research.google.com/github/henrykmichalewski/human-eval/blob/master/LLama_evals_gsm8k_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/henrykmichalewski/human-eval
!pip install -e human-eval -q

Cloning into 'human-eval'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 72 (delta 28), reused 38 (delta 13), pack-reused 9
Receiving objects: 100% (72/72), 317.58 KiB | 22.68 MiB/s, done.
Resolving deltas: 100% (28/28), done.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## The above cell clones the unlocked variant of the OpenAI HumanEval repo.

Unlocking means that the correctness code is actually executing completions. The repo contains some helper code to simplify gsm8k_python evaluations. You need to reset the runtime before moving forward.

## Loading gsm8k

In [1]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 36.2 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

In [3]:
gsm8k = load_dataset("gsm8k", "main")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [4]:
gsm8k_train, gsm8k_test = gsm8k['train'], gsm8k['test']

In [5]:
from human_eval.execution import check_gsm8k_correctness

In [6]:
check_gsm8k_correctness(gsm8k_test[5], " return 63", task_id=5)

{'task_id': 5, 'passed': False, 'result': 'failed: ', 'completion_id': None}

In [7]:
check_gsm8k_correctness(gsm8k_test[5], " return 64", task_id=5)

{'task_id': 5, 'passed': True, 'result': 'passed', 'completion_id': None}

In [8]:
check_gsm8k_correctness(gsm8k_test[5], " return 64.000001", task_id=5)

{'task_id': 5, 'passed': True, 'result': 'passed', 'completion_id': None}

## Install standard HuggingFace dependencies

In [9]:
!pip install git+https://github.com/huggingface/transformers.git@main accelerate

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-xb44becb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-xb44becb
  Resolved https://github.com/huggingface/transformers.git to commit 483861d52db59cf99219a0281695d1e7e8859218
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.33.0.dev0-py3-none-any.whl size=7604162 sha256=52d161b6878b7db1392509fc557b5b3d9f8e7f26ba55f456339050e8e4f283c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-qv86q0qo/wheels/cf/59/82/6492402e887a68975030bf8c06532260

In [10]:
!pip install sentencepiece -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.3 MB/s eta 0:00:00


In [11]:
import torch

In [12]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [13]:
device

device(type='cuda')

## Download the model

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, LlamaTokenizer, LlamaForCausalLM

# model_name = "codellama/CodeLlama-7b-Python-hf" - also a strong competitor
model_name = "codellama/CodeLlama-13b-Python-hf"

tok = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map='auto',
)
inputs = tok(["An increasing sequence: one,"], return_tensors="pt")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CodeLlamaTokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [16]:
inputs = inputs.to(model.device)
streamer = TextStreamer(tok)

output = model.generate(**inputs, streamer=streamer, max_new_tokens=20)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>An increasing sequence: one, two, three, four, five, six, seven, eight, nine, ten, eleven,


In [17]:
output

tensor([[    1,   530, 10231,  5665, 29901,   697, 29892,  1023, 29892,  2211,
         29892,  3023, 29892,  5320, 29892,  4832, 29892,  9881, 29892,  9475,
         29892, 14183, 29892,  3006, 29892, 28121, 29892]], device='cuda:0')

In [18]:
print(tok.batch_decode(output, skip_special_tokens=True))

['An increasing sequence: one, two, three, four, five, six, seven, eight, nine, ten, eleven,']


In [19]:
def generate_one_completion(prompt):
    # Convert prompt to tensors
    inputs = tok([prompt], return_tensors="pt")

    # Move tensors to the same device as the model
    inputs = inputs.to(model.device)

    # Generate output using the model
    output = model.generate(**inputs, max_new_tokens=256)

    # Decode the output and skip any special tokens
    return tok.batch_decode(output, skip_special_tokens=True)[0]

In [20]:
generate_one_completion(f"Hey there {model_name}!")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Hey there codellama/CodeLlama-13b-Python-hf!\nI'm a newbie to Python and I'm trying to make a program that will take a number and tell you if it's prime or not.\nI'm not sure if I'm doing this right, but I'm getting an error.\nI'm not sure what I'm doing wrong.\nI'm not sure if I'm doing this right, but I'm getting an error. I'm not sure what I'm doing wrong.\nYou're not doing anything wrong. The error is in the code.\nYou're not doing anything wrong. The error is in the code. You're not doing anything wrong. The error is in the code.\nYou're not doing anything wrong. The error is in the code. You're not doing anything wrong. The error is in the code. You're not doing anything wrong. The error is in the code. You're not doing anything wrong. The error is in the code. You're not doing anything wrong. The error is in the code. You're not doing anything wrong. The error is in the code. You're not doing anything wrong. The error is in the code."

## Loading/Saving samples into gdrive

In [21]:
from google.colab import drive
import os

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [22]:
path = f'/content/gdrive/My Drive/llama/{model_name}'
os.makedirs(path, exist_ok=True)
from human_eval.data import stream_jsonl, write_jsonl

### Read jsonl

In [23]:
try:
  samples = list(stream_jsonl(os.path.join(path, "samples_gsm8k.jsonl")))
except:
  samples = []

In [24]:
len(samples)

1319

## Full loop

In [25]:
from data import gsm8k_python_prompt

In [27]:
%%time
all_correct = 0
for task_id, problem in enumerate(gsm8k_test):

    # Check if task_id is less than samples length
    if task_id < len(samples):
        correctness = check_gsm8k_correctness(problem, samples[task_id]['completion'], task_id=task_id)
        correct = correctness['passed']
        all_correct += correct
        continue

    # Print Task ID
    print(f"task_id {task_id}")

    # Formulate and print the full prompt
    full_prompt = (gsm8k_python_prompt.PYTHON_PROMPT + '\n' +
                  gsm8k_python_prompt.PYTHON_TEMPLATE.format(question=problem['question']))
    completion = generate_one_completion(full_prompt)
    completion = completion[len(full_prompt):].split('\n\n')[0]
    print(completion)

    # New lines for visibility
    print("\n")
    print(problem['answer'])

    # Check for correctness of the problem solved
    correctness = check_gsm8k_correctness(problem, completion, task_id=task_id)
    correct = correctness['passed']
    all_correct += correct

    # Print out the correctness
    print(f"Correctnes: {correct}\nAll correct: {all_correct}")
    print("="*40)

    # Append the task results to the list and check if need to write to JSON file
    samples.append(dict(task_id=task_id, completion=completion))
    if task_id % 100 == 0 and task_id:
        write_jsonl(os.path.join(path, "samples_gsm8k.jsonl"), samples)

# Write all samples to JSONL file at the end
write_jsonl(os.path.join(path, "samples_gsm8k.jsonl"), samples)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


task_id 0
    eggs_per_day = 16
    eggs_for_breakfast = 3
    muffins_per_day = 4
    eggs_for_sale = eggs_per_day - eggs_for_breakfast - muffins_per_day
    dollars_per_egg = 2
    dollars_per_day = eggs_for_sale * dollars_per_egg
    return dollars_per_day


Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Correctnes: True
All correct: 1
task_id 1
    blue_bolts = 2
    white_bolts = blue_bolts / 2
    # Total bolts = blue bolts + white bolts
    total_bolts = blue_bolts + white_bolts
    return total_bolts


It takes 2/2=<<2/2=1>>1 bolt of white fiber
So the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric
#### 3


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Correctnes: True
All correct: 2
task_id 2


KeyboardInterrupt: ignored

### Write jsonl

In [ ]:
write_jsonl(os.path.join(path, "samples_gsm8k.jsonl"), samples)